<a href="https://colab.research.google.com/github/kahram-y/first-repository/blob/master/RAG/recipe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

✅ 전체 설계 요약 (RAG 최적화 관점)

1️⃣ page_content (임베딩 대상, “의미 검색용”) → 요리 핵심 맥락만 집중

-  요리명: CKG_NM
-  레시피 제목: RCP_TTL
-  요리 소개: CKG_IPDC
-  재료 목록: CKG_MTRL_CN
-  조리 방법/상황/종류: CKG_MTH_ACTO_NM / CKG_STA_ACTO_NM / CKG_KND_ACTO_NM
-  난이도, 시간, 인분: CKG_DODF_NM, CKG_TIME_NM, CKG_INBUN_NM

2️⃣ metadata (필터링·정렬·후처리용) → 검색 후 활용

-  레시피 ID: RCP_SNO
-  조회수 / 추천수 / 스크랩수: INQ_CNT / RCMM_CNT / SRAP_CNT
-  등록자: RGTR_ID
- 카테고리 태그:
-  최초 등록일: FIRST_REG_DT

In [1]:
# 핵심 라이브러리 통합 설치
!pip install pandas tqdm langchain langchain-community langchain-openai langchain-core chromadb faiss-cpu "huggingface_hub[hf_xet]"

# 모델 및 딥러닝 관련 설치
!pip install -U transformers sentence-transformers torch torchvision torchaudio

  Using cached torch-2.9.1-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (30 kB)
  Using cached torchvision-0.24.1-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (5.9 kB)
  Using cached torchaudio-2.9.1-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (6.9 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.8.93-py3-none-manylinux2010_x86_64.manylinux_2_12_x86_64.whl.metadata (1.7 kB)
  Using cached nvidia_cuda_runtime_cu12-12.8.90-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.7 kB)
  Using cached nvidia_cuda_cupti_cu12-12.8.90-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.7 kB)
  Using cached nvidia_cublas_cu12-12.8.4.1-py3-none-manylinux_2_27_x86_64.whl.metadata (1.7 kB)
  Using cached nvidia_cufft_cu12-11.3.3.83-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.7 kB)
  Using cached nvidia_curand_cu12-10.3.9.90-py3-none-manylinux_2_27_x86_64.whl.metadata (1.7 kB)
  Using cached nvidia_cusolver_cu12-11.7.3.90-py3-none-manylinux_2_2

In [3]:
from google.colab import files
uploaded = files.upload()

Saving TB_RECIPE_SEARCH-220701.csv to TB_RECIPE_SEARCH-220701.csv
Saving TB_RECIPE_SEARCH-20231130.csv to TB_RECIPE_SEARCH-20231130.csv
Saving TB_RECIPE_SEARCH_241226.csv to TB_RECIPE_SEARCH_241226.csv


In [4]:
# 컬럼 매핑 전략
TEXT_COLUMNS = [
    "RCP_TTL", "CKG_NM", "CKG_IPDC", "CKG_MTRL_CN",
    "CKG_MTH_ACTO_NM", "CKG_STA_ACTO_NM",
    "CKG_MTRL_ACTO_NM", "CKG_KND_ACTO_NM",
    "CKG_DODF_NM", "CKG_TIME_NM", "CKG_INBUN_NM"
]

**전처리 + 통합 + RAG용 Document 생성**

In [5]:
import pandas as pd
from langchain_core.documents import Document
from tqdm import tqdm

# =========================
# 1. CSV 파일 로드 및 통합
# =========================

file_paths = [
    "TB_RECIPE_SEARCH-20231130.csv",
    "TB_RECIPE_SEARCH-220701.csv",
    "TB_RECIPE_SEARCH_241226.csv"
]

dfs = []
for path in file_paths:
    try:
        # 1순위 시도: cp949 + 에러 무시
        df = pd.read_csv(path, encoding="cp949", encoding_errors="ignore")
        print(f"✅ {path} 로드 성공 (cp949)")
    except:
        # 2순위 시도: utf-8
        df = pd.read_csv(path, encoding="utf-8", encoding_errors="ignore")
        print(f"✅ {path} 로드 성공 (utf-8)")
    dfs.append(df)

recipe_df = pd.concat(dfs, ignore_index=True)

# 중복 레시피 제거 (레시피 일련번호 기준)
recipe_df = recipe_df.drop_duplicates(subset=["RCP_SNO"])

print(f"통합 레시피 수: {len(recipe_df)}")

# =========================
# 2. 결측치 처리
# =========================

recipe_df = recipe_df.fillna("정보없음")

# =========================
# 3. page_content 생성 함수
# =========================

def build_page_content(row):
    return f"""
요리명: {row['CKG_NM']}
레시피제목: {row['RCP_TTL']}

[요리 소개]
{row['CKG_IPDC']}

[재료]
{row['CKG_MTRL_CN']}

[요리 정보]
- 요리방법: {row['CKG_MTH_ACTO_NM']}
- 요리상황: {row['CKG_STA_ACTO_NM']}
- 요리종류: {row['CKG_KND_ACTO_NM']}
- 주요재료: {row['CKG_MTRL_ACTO_NM']}
- 난이도: {row['CKG_DODF_NM']}
- 소요시간: {row['CKG_TIME_NM']}
- 인분: {row['CKG_INBUN_NM']}
""".strip()

# =========================
# 4. metadata 생성 함수
# =========================

def build_metadata(row):
    return {
        "recipe_id": row["RCP_SNO"],
        "cook_name": row["CKG_NM"],
        "difficulty": row["CKG_DODF_NM"],
        "time": row["CKG_TIME_NM"],
        "ingredients_tag": row["CKG_MTRL_ACTO_NM"],
        "method": row["CKG_MTH_ACTO_NM"],
        "situation": row["CKG_STA_ACTO_NM"],
        "kind": row["CKG_KND_ACTO_NM"],
        "views": int(row["INQ_CNT"]),
        "likes": int(row["RCMM_CNT"]),
        "scraps": int(row["SRAP_CNT"]),
        "author": row["RGTR_NM"],
        "first_reg_dt": row["FIRST_REG_DT"]
    }

# =========================
# 5. LangChain Document 생성
# =========================

documents = []

for _, row in tqdm(recipe_df.iterrows(), total=len(recipe_df)):
    doc = Document(
        page_content=build_page_content(row),
        metadata=build_metadata(row)
    )
    documents.append(doc)

print(f"RAG용 Document 생성 완료: {len(documents)}")

✅ TB_RECIPE_SEARCH-20231130.csv 로드 성공 (cp949)
✅ TB_RECIPE_SEARCH-220701.csv 로드 성공 (cp949)
✅ TB_RECIPE_SEARCH_241226.csv 로드 성공 (cp949)
통합 레시피 수: 208445


100%|██████████| 208445/208445 [00:23<00:00, 8957.12it/s] 

RAG용 Document 생성 완료: 208445


**임베딩 + Chroma 벡터DB 구축**

In [9]:
# 임베딩 모델 선택
EMBEDDING_MODEL = "snunlp/KR-SBERT-V40K-klueNLI-augSTS" # 한국어 문장 의미 검색에 매우 강함, 재료/요리 설명 같은 문맥에 적합, OpenAI API 불필요

from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

# =========================
# 1. 임베딩 모델 로드
# =========================

embedding = HuggingFaceEmbeddings(
    model_name="snunlp/KR-SBERT-V40K-klueNLI-augSTS",
    model_kwargs={'device': 'cuda'}, # GPU 사용 설정
    encode_kwargs={"normalize_embeddings": True}
)

In [10]:
# =========================
# 2. Chroma DB 생성 (30분소요)
# =========================

persist_directory = "./chroma_recipe_db"

vectorstore = Chroma.from_documents(
    documents=documents,
    embedding=embedding,
    persist_directory=persist_directory,
    collection_name="korean_recipe_rag"
)

vectorstore.persist()

print("✅ Chroma 벡터 DB 생성 완료")

✅ Chroma 벡터 DB 생성 완료


/tmp/ipython-input-1779180973.py:14: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


벡터DB 로드 (서버 재시작 시)

In [ ]:
vectorstore = Chroma(
    persist_directory="./chroma_recipe_db",
    embedding_function=embedding,
    collection_name="korean_recipe_rag"
)

**Retriever 생성**

In [11]:
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={
        "k": 5
    }
)

In [12]:
# 테스트 검색
query = "두부랑 김치로 간단하게 만들 수 있는 요리"

docs = retriever.invoke(query)

for i, doc in enumerate(docs, 1):
    print(f"\n--- 결과 {i} ---")
    print(doc.page_content[:300])
    print("metadata:", doc.metadata)


--- 결과 1 ---
요리명: 두부조림양념장
레시피제목: 두부조림 / 두부요리 간단 밑반찬 만들기 ★ 간장두부조림 양념장

[요리 소개]
저희집 냉장고에 없으면 안되는게 몇개 있는데 그 중에 하나가 두부에요~ 두부는 너무 맛있어요. ^^!! 두부만 있으면 반찬없을 때 종종하게되는 간단한 밑반찬이 있는데 바로바로 두부조림이에요. 두부조림 레시피라고 부를 것도 없이 간단한데 밥한공기 뚝딱하게 만드는 반찬이에요. 밥을 비벼먹어도 되고~ 두부를 으깨서 볶아먹어도 되고~ 그냥 먹어도 되고~ 어디하나 버릴 것 없는 반찬이죠~ ^ㅡ^! 게다가 영양만점이구요~! 간단
metadata: {'recipe_id': 6857416, 'time': '15분이내', 'author': '낭랑주부', 'difficulty': '아무나', 'cook_name': '두부조림양념장', 'kind': '양념/소스/잼', 'views': 33058, 'situation': '일상', 'scraps': 199, 'method': '기타', 'ingredients_tag': '기타', 'likes': 24, 'first_reg_dt': 20160925001057}

--- 결과 2 ---
요리명: 두부조림양념장
레시피제목: 두부조림 / 두부요리 간단 밑반찬 만들기 ★ 간장두부조림 양념장

[요리 소개]
저희집 냉장고에 없으면 안되는게 몇개 있는데 그 중에 하나가 두부에요~ 두부는 너무 맛있어요. ^^!! 두부만 있으면 반찬없을 때 종종하게되는 간단한 밑반찬이 있는데 바로바로 두부조림이에요. 두부조림 레시피라고 부를 것도 없이 간단한데 밥한공기 뚝딱하게 만드는 반찬이에요. 밥을 비벼먹어도 되고~ 두부를 으깨서 볶아먹어도 되고~ 그냥 먹어도 되고~ 어디하나 버릴 것 없는 반찬이죠~ ^ㅡ^! 게다가 영양만점이구요~! 간단
metadata: {'views': 33058, 'recipe_id': 6857416, 'likes': 24, 'first_reg_dt': 20160925001057, 'time': 

**metadata 필터링 검색**

In [13]:
# Retriever 필터링 예
retriever = vectorstore.as_retriever(
    search_kwargs={
        "k": 5,
        "filter": {
            "difficulty": "초보환영"
        }
    }
)

FAISS 버전

In [ ]:
# from langchain_community.vectorstores import FAISS

# faiss_db = FAISS.from_documents(
#     documents=documents,
#     embedding=embedding
# )

# faiss_db.save_local("faiss_recipe_db")

In [ ]:
# # 로드
# faiss_db = FAISS.load_local(
#     "faiss_recipe_db",
#     embedding,
#     allow_dangerous_deserialization=True
# )

**RAG 튜닝 (요리 챗봇 특화) 시도**

In [ ]:
# ✅ 1. 재료 기반 쿼리 강화
# 사용자 질문 + "사용 가능한 재료: 두부, 김치, 양파"

# ✅ 2. 조회수 가중치 (후처리)
docs = sorted(docs, key=lambda x: x.metadata["views"], reverse=True)

# ✅ 3. Hybrid Search (추후)
# - 임베딩 검색 + 재료 키워드 매칭
# - Jaccard similarity 추가

**GPT 응답 생성용 RAG Chain**

(벡터DB에서 검색한 레시피를 바탕으로 GPT가 “요리 추천 + 조리법 + 재료 대체안”을 자연어로 생성하는 RAG Chain)

In [18]:
import os
os.environ["OPENAI_API_KEY"] = "sk-proj-"

In [ ]:
# 이미 생성한 vectorstore 사용
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5}
)

**요리 챗봇 전용 Prompt 설계** (일반 RAG와 차별화되는 부분)

🔹 Prompt 설계 포인트
- 레시피 요약
- 사용자 재료 우선 활용
- 없는 재료는 대체안 제시
- 조리 팁 & 주의사항
- 한국 요리 톤 유지

In [16]:
# Prompt Template
from langchain_core.prompts import PromptTemplate

RAG_PROMPT = PromptTemplate(
    input_variables=["context", "question"],
    template="""
당신은 한국 요리에 특화된 요리 추천 AI입니다.

아래는 사용자의 질문과 관련된 실제 레시피 정보입니다.
이 정보를 기반으로 사용자의 상황에 맞게 요리를 추천하고 설명하세요.

====================
[레시피 정보]
{context}
====================

[사용자 질문]
{question}

📌 응답 가이드:
1. 가장 적합한 요리 1~2가지를 추천
2. 각 요리에 대해
   - 요리 개요
   - 핵심 재료
   - 간단한 조리 순서
3. 사용자가 재료가 부족할 경우 가능한 재료 대체안 제시
4. 조리 시간 단축 팁 또는 맛을 살리는 팁 제공
5. 친절하고 자연스러운 한국어로 답변

응답을 시작하세요.
"""
)

LLM (GPT) 설정

In [19]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4o-mini",   # 비용/성능 균형
    temperature=0.4        # 요리는 너무 창의적일 필요 X
)

RAG Chain 구성

In [35]:
# 방법 1: LCEL(LangChain Expression Language) 방식
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.output_parsers import StrOutputParser

# 앞에서 구축한 vectorstore를 retriever로 변환
retriever = vectorstore.as_retriever()

# 체인 구성: 검색 -> 프롬프트 -> LLM -> 결과 출력 (질문과 검색 결과를 모두 유지)
rag_chain_with_source = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
).assign(
    # 이제 x에는 'context'와 'question' 키가 들어있어 프롬프트와 완벽히 매칭됩니다.
    answer= RAG_PROMPT | llm | StrOutputParser()
)

실행 테스트

In [41]:
query = "두부랑 김치가 있는데 15분 안에 만들 수 있는 쉬운 한국 요리 추천해줘"
response = rag_chain_with_source.invoke(query)

# 결과 출력
print("="*10, "GPT의 답변", "="*10)
print(response["answer"])

# 참고한 레시피(출처) 확인하기
print("\n" + "="*10, "참고 레시피 목록", "="*10)
for i, doc in enumerate(response["context"], 1):
    print(f"{i}. {doc.metadata.get('cook_name', '알 수 없음')}")
    print(f"난이도: {doc.metadata.get('difficulty', '정보 없음')}, 시간: {doc.metadata.get('time', '정보 없음')}")

========== GPT의 답변 ==========
안녕하세요! 두부와 김치가 있으시군요. 15분 안에 만들 수 있는 간단한 한국 요리로 **김두부무침**을 추천드립니다. 

### 1. 김두부무침
#### 요리 개요
김두부무침은 건강에 좋은 두부와 구운 김을 간단하게 무쳐서 만드는 요리로, 밑반찬으로도 좋고 간단한 한 끼로도 훌륭합니다.

#### 핵심 재료
- 구운 김 2장
- 두부 100g
- 간장 1/2작은술
- 통깨 약간

#### 간단한 조리 순서
1. 두부를 1cm 두께로 썰어 준비합니다.
2. 구운 김을 잘게 찢습니다.
3. 볼에 썰어 놓은 두부와 찢은 김을 넣고, 간장과 통깨를 추가합니다.
4. 재료를 잘 섞어주면 완성입니다!

#### 대체 재료
- 두부 대신 다른 콩 제품(예: 콩나물)을 사용해도 좋습니다.
- 간장 대신 소금을 조금 넣어 간을 맞춰도 괜찮습니다.

#### 조리 시간 단축 팁
- 두부는 미리 준비해 두거나, 전자레인지에 1분 정도 돌려서 데우면 더 부드럽고 빠르게 조리할 수 있습니다.

이 요리는 5분 이내로 만들 수 있어 정말 간편합니다. 두부와 김치가 함께하면 맛있는 한 끼가 되겠네요! 맛있게 드세요!

========== 참고 레시피 목록 ==========
1. 두부스테이크
난이도: 초급, 시간: 30분이내
2. 두부스테이크
난이도: 초급, 시간: 30분이내
3. 김두부무침
난이도: 아무나, 시간: 5분이내
4. 두부장아찌
난이도: 중급, 시간: 2시간이상


재료 기반 질의 강화

In [44]:
def enrich_query(user_input, ingredients=None):
    if ingredients:
        return f"{user_input}\n\n사용 가능한 재료:\n{', '.join(ingredients)}"
    return user_input

# 1. 사용자 질문 생성
query = enrich_query(
    "간단한 반찬 추천해줘",
    ingredients=["두부", "김치", "양파"]
)

# 2. 체인 실행
try:
    response = rag_chain_with_source.invoke(query)

    print("="*10, "맞춤형 레시피 추천", "="*10)
    print(response["answer"])

    print("\n" + "="*10, "참고한 레시피 출처", "="*10)
    for doc in response["context"]:
        print(f"- {doc.metadata['cook_name']} ({doc.metadata['difficulty']})")

except Exception as e:
    print(f"실행 중 오류 발생: {e}")

========== 맞춤형 레시피 추천 ==========
간단한 반찬으로 두부를 활용한 두 가지 요리를 추천해드릴게요!

### 1. 두부조림
#### 요리 개요
두부조림은 간단하면서도 맛있는 밑반찬으로, 양념간장을 사용해 두부를 조리하여 깊은 맛을 낼 수 있습니다. 초스피드로 만들 수 있어 바쁜 날에도 안성맞춤입니다.

#### 핵심 재료
- 두부 1모
- 양파 1/4개
- 양념간장 (국간장 2큰술, 양파청 1.5큰술, 고춧가루 1작은술, 다진파 1큰술, 깨소금 1큰술)
- 물 또는 육수 1~2큰술
- 다진파, 통깨 (선택)

#### 간단한 조리 순서
1. 두부는 적당한 크기로 썰고, 양파는 채 썰어 준비합니다.
2. 팬에 식용유와 들기름을 두르고 양파를 볶아 향을 낸 후, 두부를 넣고 살짝 구워줍니다.
3. 양념간장과 물 또는 육수를 넣고 중불에서 조리합니다.
4. 마지막에 다진파와 통깨를 뿌려서 마무리합니다.

#### 대체 재료
양파 대신 대파나 청양고추를 사용해도 좋습니다. 양념간장은 미리 만들어 두면 여러 요리에 활용할 수 있습니다.

#### 조리 시간 단축 팁
양념간장을 미리 만들어 두면 조리 시간이 단축됩니다. 또한, 두부를 미리 구워두면 조리 시간이 더 줄어듭니다.

---

### 2. 두부야채부침
#### 요리 개요
두부야채부침은 두부와 다양한 채소를 함께 부쳐내는 간단한 반찬입니다. 바삭한 식감이 매력적이며, 양파와 당근을 활용해 영양도 챙길 수 있습니다.

#### 핵심 재료
- 두부 1모
- 양파 1/2개
- 당근 1/4개
- 달걀 1개
- 밀가루 2큰술

#### 간단한 조리 순서
1. 두부는 적당한 두께로 썰고, 양파와 당근도 채 썰어줍니다.
2. 볼에 두부, 양파, 당근, 달걀, 밀가루를 넣고 잘 섞어줍니다.
3. 팬에 기름을 두르고, 반죽을 한 스푼씩 떠서 부쳐줍니다. 양면이 노릇하게 구워지면 완성입니다.

#### 대체 재료
당근 대신 다른 채소(예: 호박, 청경채)를 사용해도 좋습니다.

#### 조리 시간 단축 팁
두

품질 향상 시도

In [ ]:
# ✅ 1. 조회수 기반 재정렬
docs = response["source_documents"]
docs = sorted(docs, key=lambda d: d.metadata["views"], reverse=True)

# ✅ 2. 난이도 자동 필터
retriever.search_kwargs["filter"] = {"difficulty": "초보환영"}

# ✅ 3. Context 길이 제어
context = "\n\n".join(doc.page_content[:500] for doc in docs[:3])